In [61]:
import requests
import time
import json
from urllib.parse import urljoin, urlencode
import hmac
import hashlib
import hmac
import hashlib
import pandas as pd
import numpy as np

In [22]:
proxies = {'http': 'http://127.0.0.1:1086',
           'https': 'http://127.0.0.1:1086'}

base_url = "https://vapi.binance.com"
api_key = "8YbN0qDgaPhYQCQjW8v2v3JOWNHoTjOZgLkXsF1MeT3BY4Lu1TzVrPCd3SQRSBuf"
api_secret = "GNStPoGYiM5fb03fJUx0uSnltJUK8IPTF8f8nk1LcHl6mqsVHDuEs9AIO0GWHUCe"
headers = {
    'X-MBX-APIKEY': api_key
}

In [8]:
class BinanceException(Exception):
    def __init__(self, status_code, data):
        self.status_code = status_code
        if data:
            self.code = data['code']
            self.msg = data['msg']
        else:
            self.code = None
            self.msg = None
        message = f"{status_code} [{self.code}] {self.msg}"

        super().__init__(message)

In [23]:
path = "/vapi/v1/ping"
url = urljoin(base_url, path)
r = requests.get(url, headers=headers, proxies=proxies)
r.text

'{"code":0,"msg":"success"}'

In [55]:
time.time()

1648189263.655469

In [101]:
class binance_option_api():
    def __init__(self, api_key, api_secret, proxies):
        self.api_key = api_key
        self.api_secret = api_secret
        self.headers = {
            'X-MBX-APIKEY': api_key
        }
        self.proxies = proxies
        self.base_url = "https://vapi.binance.com"

    def get_option_info(self):
        path = "/vapi/v1/optionInfo"
        url = urljoin(self.base_url, path)
        params = None
        r = requests.get(url=url, headers=self.headers, params=params, proxies=self.proxies)
        if r.status_code == 200:
            return r.json()['data']
        else:
            raise BinanceException(status_code=r.status_code, data=r.json())

    def get_exchange_info(self):
        path = "/vapi/v1/exchangeInfo"
        url = urljoin(self.base_url, path)
        params = None
        r = requests.get(url=url, headers=self.headers, params=params, proxies=self.proxies)
        if r.status_code == 200:
            return r.json()['data']
        else:
            raise BinanceException(status_code=r.status_code, data=r.json())

    def get_spot_price(self, underlying: str):
        path = "/vapi/v1/index"
        url = urljoin(self.base_url, path)
        params = {"underlying": underlying}
        r = requests.get(url=url, headers=self.headers, params=params, proxies=self.proxies)
        if r.status_code == 200:
            return r.json()['data']
        else:
            raise BinanceException(status_code=r.status_code, data=r.json())

    def get_latest_24h_price(self, symbol: str):
        path = "/vapi/v1/ticker"
        url = urljoin(self.base_url, path)
        params = {"symbol": symbol}
        r = requests.get(url=url, headers=self.headers, params=params, proxies=self.proxies)
        if r.status_code == 200:
            return r.json()['data']
        else:
            raise BinanceException(status_code=r.status_code, data=r.json())

    def get_latest_24h_price(self, symbol: str):
        path = "/vapi/v1/ticker"
        url = urljoin(self.base_url, path)
        params = {"symbol": symbol}
        r = requests.get(url=url, headers=self.headers, params=params, proxies=self.proxies)
        if r.status_code == 200:
            return r.json()['data']
        else:
            raise BinanceException(status_code=r.status_code, data=r.json())

    def get_mark_price(self, symbol: str):
        path = "/vapi/v1/mark"
        url = urljoin(self.base_url, path)
        params = {"symbol": symbol}
        r = requests.get(url=url, headers=self.headers, params=params, proxies=self.proxies)
        if r.status_code == 200:
            return r.json()['data']
        else:
            raise BinanceException(status_code=r.status_code, data=r.json())

    def get_depth(self, symbol: str, limit: int=10):

        path = "/vapi/v1/depth"
        url = urljoin(self.base_url, path)
        params = {"symbol": symbol, "limit": limit}
        r = requests.get(url=url, headers=self.headers, params=params, proxies=self.proxies)
        if r.status_code == 200:
            return r.json()['data']
        else:
            raise BinanceException(status_code=r.status_code, data=r.json())

    def get_k_line(self, symbol: str, interval: str, start_time: str, end_time: str, limit: int):
        path = "/vapi/v1/klines"
        url = urljoin(self.base_url, path)
        params = {"symbol": symbol, 'interval': interval, 'startTime': start_time, 'endTime': end_time, "limit": limit}
        r = requests.get(url=url, headers=self.headers, params=params, proxies=self.proxies)
        if r.status_code == 200:
            return r.json()['data']
        else:
            raise BinanceException(status_code=r.status_code, data=r.json())

    def get_latest_trades(self, symbol: str, limit: int=10):
        path = "/vapi/v1/trades"
        url = urljoin(self.base_url, path)
        params = {"symbol": symbol, "limit": limit}
        r = requests.get(url=url, headers=self.headers, params=params, proxies=self.proxies)
        if r.status_code == 200:
            return r.json()['data']
        else:
            raise BinanceException(status_code=r.status_code, data=r.json())

    def make_single_order(self, **params):
        path = "/vapi/v1/order"
        url = urljoin(self.base_url, path)
        query_string = urlencode(params)
        params['signature'] = hmac.new(self.api_secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
        r = requests.get(url=url, headers=self.headers, params=params, proxies=self.proxies)
        if r.status_code == 200:
            return r.json()['data']
        else:
            raise BinanceException(status_code=r.status_code, data=r.json())

    def get_option_left_days(self, symbol: str):

        temp = pd.DataFrame(self.get_option_info())
        expire = temp['expiryDate'].loc[temp.symbol == symbol].values[0] / 1000
        now = round(time.time())
        left_days = (expire - now) / (3600 * 24)

        return round(left_days, 3)

In [102]:
client = binance_option_api(api_key=api_key, api_secret=api_secret, proxies=proxies)

In [106]:
left_days = client.get_option_left_days(symbol='BTC-220429-45000-C')

In [88]:
temp = client.get_latest_24h_price(symbol='BTC-220429-45000-C')

In [91]:
temp = client.get_option_info()

In [92]:
temp

[{'id': 3363,
  'contractId': 1,
  'underlying': 'BTCUSDT',
  'quoteAsset': 'USDT',
  'symbol': 'BTC-220325-38000-P',
  'unit': '1',
  'minQty': '0.0002',
  'maxQty': '10000',
  'priceScale': 2,
  'quantityScale': 4,
  'side': 'PUT',
  'leverage': '9999',
  'strikePrice': '38000',
  'makerFeeRate': '0.0003',
  'takerFeeRate': '0.0003',
  'initialMargin': '0.15',
  'autoReduceMargin': '0.075',
  'maintenanceMargin': '0.02',
  'minInitialMargin': '0.1',
  'minAutoReduceMargin': '0.05',
  'minMaintenanceMargin': '0.013',
  'expiryDate': 1648195200000,
  'filters': [{'filterType': 'PRICE_FILTER',
    'minPrice': '0.02',
    'maxPrice': '1766.06',
    'tickSize': '0.01'},
   {'filterType': 'LOT_SIZE',
    'minQty': '0.0002',
    'maxQty': '10000',
    'stepSize': '0.0001'}]},
 {'id': 3025,
  'contractId': 1,
  'underlying': 'BTCUSDT',
  'quoteAsset': 'USDT',
  'symbol': 'BTC-220325-55000-C',
  'unit': '1',
  'minQty': '0.0002',
  'maxQty': '10000',
  'priceScale': 2,
  'quantityScale': 4,
 

In [85]:
left_days = (df['expiryDate'].loc[df.symbol == 'BTC-220429-45000-C'].values[0] / 1000 - round(time.time())) / (3600 * 24)

35.06378472222222

In [53]:
temp[0]['bidPrice']

'3038.51'

In [54]:
temp[0]['askPrice']

'3201.45'

In [ ]:
""

In [48]:
client.get_mark_price(symbol='BTC-220429-45000-C')

{'code': 0,
 'msg': 'success',
 'data': [{'symbol': 'BTC-220429-45000-C',
   'markPrice': '3101.22',
   'bidIV': '0.6274',
   'askIV': '0.6504',
   'delta': '0.50055',
   'theta': '-49.71587',
   'gamma': '0.00005',
   'vega': '54.58233',
   'volatility': '0.63889',
   'highPriceLimit': '4251.72',
   'lowPriceLimit': '1950.72'}]}

In [ ]:
def get_option_info():
    path = "/vapi/v1/optionInfo"
    url = urljoin(base_url, path)
